In [ ]:
!pip install --upgrade python-dotenv
!pip install langchain_community
%pip install --upgrade --quiet  wikipedia
%pip install --upgrade --quiet  duckduckgo-search

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 57.0 MB/s eta 0:00:00


In [ ]:
!pip install langchain langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.1 MB/s eta 0:00:00


In [ ]:
import os
from dotenv import load_dotenv
import pickle
# from openai import OpenAI
import IPython

import requests

import nltk
nltk.download('punkt')
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
import numpy as np

# dotenv_path = '/content/drive/MyDrive/consult-proj/.env'
# load_dotenv(dotenv_path)

# openai_api_key = os.getenv("OPENAI_API_KEY")
# rapid_api_key = os.getenv("RAPID_API_KEY")
# # rapid_api_key

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from langchain_community.retrievers import WikipediaRetriever
from langchain.chains import ConversationalRetrievalChain
from langchain_openai import ChatOpenAI
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

### LinkedIn

In [ ]:
path = '/content/drive/MyDrive/consult-proj/kubig_consult_proj/mk/industry_graph/data/company_info.pkl'

with open(path, 'rb') as f:
    company = pickle.load(f)

In [ ]:
def similar_company(target):
    sim_company = {}
    for company in target['similar_pages']:
        sim_company[company['title']] = company['industry']
    return sim_company

def company_info_dict(target):
    return {
    'company_name' : target['company_name'],
    'description' : target['description'],
    'company_size' : target['company_size'],
    'industries' : target['industries'],
    'specialties' : target['specialties'],
    'similar_company' : similar_company(target)
    }

# 단어 임베딩 불러오기
word_embeddings = {}

with open("/content/drive/MyDrive/consult-proj/glove.6B.100d.txt", 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        word_embeddings[word] = vector

def similarity_company(target_company):

    description = target_company['description']
    keywords = target_company['specialties']

    description_tokens = word_tokenize(description.lower())
    description_vectors = [word_embeddings[word] for word in description_tokens if word in word_embeddings]

    sim_dict = {}

    for keyword in keywords:
        # 토큰화 및 토큰 벡터화
        keyword_tokens = word_tokenize(keyword.lower())
        keyword_vectors = [word_embeddings[word] for word in keyword_tokens if word in word_embeddings]

        # 벡터 평균 계산
        description_vector_avg = np.mean(description_vectors, axis=0)
        keyword_vector_avg = np.mean(keyword_vectors, axis=0)

        # 코사인 유사도 계산
        similarity = cosine_similarity([description_vector_avg], [keyword_vector_avg])
        sim_dict[keyword] = similarity[0][0]
        # print(f"{keyword} : {similarity[0][0]}")

        sim_dict = {key : value for key, value in sim_dict.items() if value >= 0.5}

    return sim_dict

# 딕셔너리 값과 리스트 요소 간의 유사도 측정
def similar_company_similarity(sim_copmany, sim_dict):
    sim_comp_dict = {}
    company_spec = list(sim_dict.keys())

    for company, industry in sim_company.items():
        similarities = {}
        # 딕셔너리 값의 벡터 계산
        company_vector = np.mean([word_embeddings[word] for word in industry.lower().split() if word in word_embeddings], axis=0)
        for keyword in company_spec:
            # 리스트 요소의 벡터 계산
            keyword_vector = np.mean([word_embeddings[word] for word in keyword.lower().split() if word in word_embeddings], axis=0)
            # 코사인 유사도 계산
            similarity_score = cosine_similarity([company_vector], [keyword_vector])[0][0]
            # 딕셔너리에 유사도 저장
            similarities[keyword] = similarity_score
        sim_comp_dict[company] = similarities

    return sim_comp_dict

def similar_company_final(sim_company, sim_dict):

    final_comp_dict = {}
    company_spec = list(sim_dict.keys())

    for company, industry in sim_company.items():
        similarities = {}
        # 딕셔너리 값의 벡터 계산
        score = 0
        company_vector = np.mean([word_embeddings[word] for word in industry.lower().split() if word in word_embeddings], axis=0)
        for keyword in company_spec:
            # 리스트 요소의 벡터 계산
            keyword_vector = np.mean([word_embeddings[word] for word in keyword.lower().split() if word in word_embeddings], axis=0)
            # 코사인 유사도 계산
            similarity_score = cosine_similarity([company_vector], [keyword_vector])[0][0]
            # 딕셔너리에 유사도 저장
            # similarities[keyword] = similarity_score * sim_dict[keyword]

            # 코사인 유사도 끼리의 내적을 summation
            score += similarity_score * sim_dict[keyword]

        final_comp_dict[company] = score

        sorted_data = sorted(final_comp_dict.items(), key=lambda x: x[1], reverse=True)
        sorted_dict = {item[0]: item[1] for item in sorted_data}

    return sorted_dict

In [ ]:
target = company['results'][0]

target_company = company_info_dict(target)
print(target_company)

sim_company = target_company['similar_company']
print(sim_company)

{'company_name': 'Microsoft', 'description': "Every company has a mission. What's ours? To empower every person and every organization to achieve more. We believe technology can and should be a force for good and that meaningful innovation contributes to a brighter world in the future and today. Our culture doesn’t just encourage curiosity; it embraces it. Each day we make progress together by showing up as our authentic selves. We show up with a learn-it-all mentality. We show up cheering on others, knowing their success doesn't diminish our own. We show up every day open to learning our own biases, changing our behavior, and inviting in differences. Because impact matters. \n\nMicrosoft operates in 190 countries and is made up of more than 220,000 passionate employees worldwide.", 'company_size': '10,001+ employees', 'industries': 'Software Development', 'specialties': ['Business Software', 'Developer Tools', 'Home & Educational Software', 'Tablets', 'Search', 'Advertising', 'Servers

In [ ]:
# microsoft description과 microsoft keyword 간의 cosine similarity
sim_dict = similarity_company(target_company)
sim_dict

{'Business Software': 0.67401576,
 'Home & Educational Software': 0.72484416,
 'Search': 0.5987896,
 'Advertising': 0.5249061,
 'Windows Operating System': 0.6668994,
 'Windows Applications & Platforms': 0.5649132,
 'Future of Work': 0.9047775,
 'Artificial Intelligence': 0.5658074,
 'Machine Learning': 0.6642258,
 'Mixed Reality': 0.6871526,
 'Virtual Reality': 0.6077376,
 'IT Professional': 0.8825605}

In [ ]:
# google industry(similar company)와 microsoft keyword 간의 cosine similarity
similar_company_similarity(sim_company, sim_dict)['Google']

{'Business Software': 0.9019533,
 'Home & Educational Software': 0.8337043,
 'Search': 0.57643837,
 'Advertising': 0.5301915,
 'Windows Operating System': 0.7488832,
 'Windows Applications & Platforms': 0.71754587,
 'Future of Work': 0.66631,
 'Artificial Intelligence': 0.5641777,
 'Machine Learning': 0.70104784,
 'Mixed Reality': 0.47129786,
 'Virtual Reality': 0.54647493,
 'IT Professional': 0.6130804}

In [ ]:
final = similar_company_final(sim_company, sim_dict)
final

{'IBM': 5.566595017910004,
 'Oracle': 5.566595017910004,
 'Google': 5.325263023376465,
 'Amazon': 5.325263023376465,
 'Meta': 5.325263023376465,
 'LinkedIn': 5.325263023376465,
 'Deloitte': 5.274071037769318,
 'Accenture': 5.274071037769318,
 'Apple': 4.908330321311951,
 'Netflix': 4.31483268737793}

### LinkedIn_v2 (+Wikipedia)

In [ ]:
query = 'microsoft'

retriever = WikipediaRetriever()
docs = retriever.get_relevant_documents(query=query)
docs[0].metadata['summary']

'Microsoft Corporation is an American multinational corporation and technology company headquartered in Redmond, Washington. Microsoft\'s best-known software products are the Windows line of operating systems, the Microsoft 365 suite of productivity applications, and the Edge web browser. Its flagship hardware products are the Xbox video game consoles and the Microsoft Surface lineup of touchscreen personal computers. Microsoft ranked No. 14 in the 2022 Fortune 500 rankings of the largest United States corporations by total revenue; and it was the world\'s largest software maker by revenue in 2022 according to Forbes Global 2000. It is considered one of the Big Five American information technology companies, alongside Alphabet (parent company of Google), Amazon, Apple, and Meta (parent company of Facebook).\nMicrosoft was founded by Bill Gates and Paul Allen on April 4, 1975, to develop and sell BASIC interpreters for the Altair 8800. It rose to dominate the personal computer operating

In [ ]:
import wikipedia

def get_wikipedia_summary(query):
    try:
        summary = wikipedia.summary(query)
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        # Handle disambiguation pages
        options = e.options
        return f"Disambiguation page. Options: {', '.join(options)}"
    except wikipedia.exceptions.PageError:
        return "Page not found."

query = "microsoft"
summary = get_wikipedia_summary(query)
print(summary)

Microsoft Corporation is an American multinational corporation and technology company headquartered in Redmond, Washington. Microsoft's best-known software products are the Windows line of operating systems, the Microsoft 365 suite of productivity applications, and the Edge web browser. Its flagship hardware products are the Xbox video game consoles and the Microsoft Surface lineup of touchscreen personal computers. Microsoft ranked No. 14 in the 2022 Fortune 500 rankings of the largest United States corporations by total revenue; and it was the world's largest software maker by revenue in 2022 according to Forbes Global 2000. It is considered one of the Big Five American information technology companies, alongside Alphabet (parent company of Google), Amazon, Apple, and Meta (parent company of Facebook).
Microsoft was founded by Bill Gates and Paul Allen on April 4, 1975, to develop and sell BASIC interpreters for the Altair 8800. It rose to dominate the personal computer operating sys

In [ ]:
def similarity_company_v2(target_company, threshold=0.5):

    # WikipediaRetrieve 결과로 target_company description 교체
    retriever = WikipediaRetriever()
    description = retriever.get_relevant_documents(query=query)[0].metadata['summary']
    # description = get_wikipedia_summary(query=target_company['company_name'])


    keywords = target_company['specialties']

    description_tokens = word_tokenize(description.lower())
    description_vectors = [word_embeddings[word] for word in description_tokens if word in word_embeddings]

    sim_dict = {}

    for keyword in keywords:
        # 토큰화 및 토큰 벡터화
        keyword_tokens = word_tokenize(keyword.lower())
        keyword_vectors = [word_embeddings[word] for word in keyword_tokens if word in word_embeddings]

        # 벡터 평균 계산
        description_vector_avg = np.mean(description_vectors, axis=0)
        keyword_vector_avg = np.mean(keyword_vectors, axis=0)

        # 코사인 유사도 계산
        similarity = cosine_similarity([description_vector_avg], [keyword_vector_avg])
        sim_dict[keyword] = similarity[0][0]
        # print(f"{keyword} : {similarity[0][0]}")

        sim_dict = {key : value for key, value in sim_dict.items() if value >= threshold}

    return sim_dict

# 딕셔너리 값과 리스트 요소 간의 유사도 측정
def similar_company_similarity_v2(sim_copmany, sim_dict):
    sim_comp_dict = {}
    company_spec = list(sim_dict.keys())

    for company, industry in sim_company.items():
        similarities = {}
        # WikipediaRetrieve 결과로 similar_comapny industry를 대체
        retriever = WikipediaRetriever()
        description = retriever.get_relevant_documents(query=company)[0].metadata['summary']
        # 딕셔너리 값의 벡터 계산
        company_vector = np.mean([word_embeddings[word] for word in description.lower().split() if word in word_embeddings], axis=0)

        for keyword in company_spec:
            # 리스트 요소의 벡터 계산
            keyword_vector = np.mean([word_embeddings[word] for word in keyword.lower().split() if word in word_embeddings], axis=0)
            # 코사인 유사도 계산
            similarity_score = cosine_similarity([company_vector], [keyword_vector])[0][0]
            # 딕셔너리에 유사도 저장
            similarities[keyword] = similarity_score
        sim_comp_dict[company] = similarities

    return sim_comp_dict

def similar_company_final_v2(sim_company, sim_dict):
    # similar_company_similarity_v2 수정 반영

    final_comp_dict = {}
    company_spec = list(sim_dict.keys())

    for company, industry in sim_company.items():
        similarities = {}
        score = 0

        # WikipediaRetrieve 결과로 similar_comapny industry를 대체
        retriever = WikipediaRetriever()
        description = retriever.get_relevant_documents(query=company)[0].metadata['summary']
        # 딕셔너리 값의 벡터 계산
        company_vector = np.mean([word_embeddings[word] for word in description.lower().split() if word in word_embeddings], axis=0)

        for keyword in company_spec:
            # 리스트 요소의 벡터 계산
            keyword_vector = np.mean([word_embeddings[word] for word in keyword.lower().split() if word in word_embeddings], axis=0)
            # 코사인 유사도 계산
            similarity_score = cosine_similarity([company_vector], [keyword_vector])[0][0]

            # 코사인 유사도 끼리의 내적을 summation
            score += similarity_score * sim_dict[keyword]


        final_comp_dict[company] = score

        sorted_data = sorted(final_comp_dict.items(), key=lambda x: x[1], reverse=True)
        sorted_dict = {item[0]: item[1] for item in sorted_data}

    return sorted_dict

In [ ]:
# microsoft description과 microsoft keyword 간의 cosine similarity
"""
  microsoft description >> microsoft wikipedia result 로 교체
"""
target = company['results'][0]
target_company = company_info_dict(target)
print(target_company)
print()

sim_dict = similarity_company_v2(target_company, threshold=0.5)
sim_dict

{'company_name': 'Microsoft', 'description': "Every company has a mission. What's ours? To empower every person and every organization to achieve more. We believe technology can and should be a force for good and that meaningful innovation contributes to a brighter world in the future and today. Our culture doesn’t just encourage curiosity; it embraces it. Each day we make progress together by showing up as our authentic selves. We show up with a learn-it-all mentality. We show up cheering on others, knowing their success doesn't diminish our own. We show up every day open to learning our own biases, changing our behavior, and inviting in differences. Because impact matters. \n\nMicrosoft operates in 190 countries and is made up of more than 220,000 passionate employees worldwide.", 'company_size': '10,001+ employees', 'industries': 'Software Development', 'specialties': ['Business Software', 'Developer Tools', 'Home & Educational Software', 'Tablets', 'Search', 'Advertising', 'Servers

{'Business Software': 0.7680514,
 'Developer Tools': 0.56962705,
 'Home & Educational Software': 0.7939272,
 'Search': 0.61108184,
 'Advertising': 0.57576287,
 'Windows Operating System': 0.7576927,
 'Windows Applications & Platforms': 0.6649166,
 'Cloud Computing': 0.50872856,
 'Future of Work': 0.8764821,
 'Artificial Intelligence': 0.57538,
 'Machine Learning': 0.65275466,
 'Mixed Reality': 0.6489361,
 'Virtual Reality': 0.58589995,
 'IT Professional': 0.8582895}

In [ ]:
# google industry(similar company)와 microsoft keyword 간의 cosine similarity
"""
  google industry >> google wikipedia result 로 교체
"""
sim_company = target_company['similar_company']
print(sim_company)
print()

similar_company_similarity_v2(sim_company, sim_dict)

{'Google': 'Software Development', 'Amazon': 'Software Development', 'Apple': 'Computers and Electronics Manufacturing', 'IBM': 'IT Services and IT Consulting', 'Meta': 'Software Development', 'Deloitte': 'Business Consulting and Services', 'Netflix': 'Entertainment Providers', 'LinkedIn': 'Software Development', 'Oracle': 'IT Services and IT Consulting', 'Accenture': 'Business Consulting and Services'}



/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave diff

{'Google': {'Business Software': 0.7907332,
  'Developer Tools': 0.6037532,
  'Home & Educational Software': 0.8231952,
  'Search': 0.65985554,
  'Advertising': 0.60510796,
  'Windows Operating System': 0.76282316,
  'Windows Applications & Platforms': 0.6961033,
  'Cloud Computing': 0.55616814,
  'Future of Work': 0.8873499,
  'Artificial Intelligence': 0.6109588,
  'Machine Learning': 0.69910896,
  'Mixed Reality': 0.662857,
  'Virtual Reality': 0.6215099,
  'IT Professional': 0.8576421},
 'Amazon': {'Business Software': 0.7606787,
  'Developer Tools': 0.5547945,
  'Home & Educational Software': 0.7961292,
  'Search': 0.63587505,
  'Advertising': 0.6056546,
  'Windows Operating System': 0.7311183,
  'Windows Applications & Platforms': 0.64388025,
  'Cloud Computing': 0.5155651,
  'Future of Work': 0.8909542,
  'Artificial Intelligence': 0.5883682,
  'Machine Learning': 0.6533588,
  'Mixed Reality': 0.66698486,
  'Virtual Reality': 0.5999801,
  'IT Professional': 0.86745644},
 'Apple'

In [ ]:
final = similar_company_final_v2(sim_company, sim_dict)
final

/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
/usr/local/lib/python3.10/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave diff

{'Google': 5.947384059429169,
 'Amazon': 5.8012948632240295,
 'Meta': 5.748520880937576,
 'LinkedIn': 5.720881342887878,
 'IBM': 5.676479905843735,
 'Netflix': 5.6651842296123505,
 'Deloitte': 5.572058469057083,
 'Accenture': 5.560253471136093,
 'Oracle': 5.554120272397995,
 'Apple': 5.481934815645218}

더 나아가서 yahoo finance 같은데서 sector 정보 받아와 microsoft와 같은 sector의 기업들의 wikipedia 정보 받아와 전수조사도 가능

### DuckDuckGo

In [ ]:
search = DuckDuckGoSearchRun()
search = DuckDuckGoSearchResults(backend="news")
query = 'microsoft'

In [ ]:
search.run(f'explan about {query} company')

'Microsoft\'s Mission Statement. Microsoft\'s mission is " to empower every person and every organization on the planet to achieve more. " This mission statement shows that the business focuses on the empowerment of people and organizations, through the company\'s information technology goods and services. The following components are ... A company\'s corporate structure refers to the anatomy and arrangement of the organization and its components. In Microsoft\'s case, the organizational structure stresses the importance of business output. Considering the company\'s dominance in the IBM PC-compatible operating system market, this company structure helps maintain competitive ... Microsoft is a multinational computer technology corporation. Microsoft was founded on April 4, 1975, by Bill Gates and Paul Allen in Albuquerque, New Mexico. Its current best-selling products are the Microsoft Windows operating system; Microsoft Office, a suite of productivity software; Xbox, a line of enterta

In [ ]:
search.run(query)

"[snippet: Microsoft is making a $1.5 billion investment into United Arab Emirates-based AI company, Group 42 Holding (G42), that will bring its AI technology to the UAE and other countries, it said Tuesday., title: Microsoft's new AI play is a $1.5 billion investment in an Abu Dhabi firm with China ties, link: https://www.msn.com/en-us/money/other/microsofts-new-ai-play-is-a-15-billion-investment-in-an-abu-dhabi-firm-with-china-ties/ar-BB1lIxTp, date: 2024-04-16T13:13:02+00:00, source: Quartz on MSN.com], [snippet: As the Gulf region gains growing strategic importance in the tech war between the U.S. and China, Microsoft makes a big move into one of its oil-rich countries. On Monday evening, Microsoft announced a $1., title: Microsoft's $1.5B funding for G42 signals growing US-China rift, link: https://news.yahoo.com/microsofts-1-5b-check-g42-084415553.html, date: 2024-04-16T10:01:00+00:00, source: YAHOO!News], [snippet: Data security company Rubrik has set terms for its initial publi

In [ ]:
wrapper = DuckDuckGoSearchAPIWrapper(region="us-en", time="d", max_results=2)
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news")
search.run(query)

"[snippet: If you have a Windows 10 computer, Microsoft will offer you a free upgrade to Windows 11. Provided the hardware of the Windows 10 computer is compatible, otherwise Windows 11 will refuse the ..., title: Windows PCs are now getting the free upgrade to Windows 11, link: https://www.pcworld.com/article/2304543/more-windows-pcs-are-now-getting-the-free-upgrade-to-windows-11-the-reason.html], [snippet: MSFT. (Bloomberg) -- Microsoft Corp. will invest $1.5 billion in the United Arab Emirates' top artificial intelligence firm, G42, after the Abu Dhabi-based company worked out an unusual deal ..., title: Microsoft Invests $1.5 Billion in UAE's G42 in Pivot From China, link: https://finance.yahoo.com/news/microsoft-invests-1-5-billion-041144875.html], [snippet: After rocketing higher in 2023, Microsoft (NASDAQ: MSFT) stock has continued to hit new highs in 2024, currently up 10% year to date. Morgan Stanley analyst Keith Weiss sees even further upside as ..., title: Microsoft Stock H

### RAG

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun, DuckDuckGoSearchResults
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.callbacks import get_openai_callback

In [ ]:
## Provide "the latest context information" from web search
template = """Answer the question based on context.

Question: {question}
Context: {context}
Answer:"""


prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-3.5-turbo")
parser = StrOutputParser()

# RunnablePassthrough() 는 그 중간에 위치하여 어떤 변환도 가하지 않고 그대로 전달하는 역할 >> 데이터가 특별한 가공 없이 전달되어야 할 때 유용하게 사용
chain = (
    {"question": RunnablePassthrough(), "context": RunnablePassthrough()}
    | prompt
    | model
    | parser
)

In [ ]:
search = DuckDuckGoSearchResults(backend="news")
search_result = search.run("OpenAI funding, partners information")
search_result

'[snippet: OpenAI CEO Sam Altman and former Apple design chief Jony Ive have officially teamed up to design an AI-powered personal device and are, title: Jony Ive and OpenAI\'s Sam Altman Seeking Funding for Personal AI Device, link: https://www.macrumors.com/2024/04/05/jony-ive-sam-altman-ai-device/, date: 2024-04-05T22:35:00+00:00, source: MacRumors], [snippet: Microsoft and OpenAI have joined forces for an ambitious project to revolutionize artificial intelligence (AI) capabilities. This collaboration entails the development of a cutting-edge data center alongside an AI supercomputer dubbed "Stargate,, title: Microsoft and OpenAI Collaborate on $100 Billion Supercomputer Project, link: https://www.msn.com/en-us/news/technology/microsoft-and-openai-collaborate-on-100-billion-supercomputer-project/ar-BB1kZcgt, date: 2024-04-03T06:33:56+00:00, source: Cryptopolitan on MSN.com], [snippet: Elon Musk\'s AI startup xAI targets $3-4 billion in funding with an $18 billion valuation, competin

In [ ]:
question = "Create key points about OpenAI such as funding, partners information."
search = DuckDuckGoSearchResults(backend="news")

with get_openai_callback() as cb:
  result = chain.invoke({"question": question, "context": search.run(question)})
  print(result)
  print("---" * 10)
  print(cb)

- Adobe is exploring a partnership with OpenAI to incorporate AI video tools.
- OpenAI has opened a new office in Tokyo, its first in Asia.
- OpenAI dismissed two researchers for leaking classified information.
- OpenAI's voice cloning AI model only requires a 15-second sample to work.
------------------------------
Tokens Used: 1022
	Prompt Tokens: 962
	Completion Tokens: 60
Successful Requests: 1
Total Cost (USD): $0.0015630000000000002


In [ ]:
question = "Tell me about Talyor Swift boyfriend"
wrapper = DuckDuckGoSearchAPIWrapper(region="us-en", time="d", max_results=2)
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source="news")
search.run(question)

'[snippet: Courtney Love has a new radio show about women in music, but she is not much of a fan of several of the biggest female artists of our time. Love, the frontwoman for the band Hole, this month ..., title: Courtney Love thinks Taylor Swift is \'not important\' and has some ..., link: https://edition.cnn.com/2024/04/16/entertainment/courtney-love-taylor-swift-beyonce/index.html], [snippet: In an interview with The Evening Standard on Friday (April 12) to promote her upcoming BBC Sounds series, named Courtney Love\'s Women, she revealed what she thought of the \'Shake It Off\' singer.. Speaking her thoughts on various female artists in the music industry, she told the outlet: "Taylor is not important. She might be a safe space for girls, and she\'s probably the Madonna of now, but ..., title: Courtney Love says Taylor Swift is \'not important\' or \'interesting\', link: https://www.unilad.com/music/news/courtney-love-calls-out-taylor-swift-370278-20240416], [snippet: "Taylor is n

In [ ]:
with get_openai_callback() as cb:
  result = chain.invoke({"question": question, "context": search.run(question)})
  print(result)
  print("---" * 10)
  print(cb)

Taylor Swift's boyfriend is Travis Kelce.
------------------------------
Tokens Used: 941
	Prompt Tokens: 932
	Completion Tokens: 9
Successful Requests: 1
Total Cost (USD): $0.0014160000000000002
